### My idea is to take a user input for particular city name or country name, then anakyze the country's venue data and venue-category data using k-means clustering we can show user the map of clusters in their city, and according to those clusters we can provide the results about the city, that what are the local people upto the most! It's an interesting way of knowing any city in the world, or finding some new characterstics of your own city.

### let's start by finding the coordinates of user's city using Geopy

In [1]:
import pandas as pd
import numpy as np
import requests
import folium
from geopy.geocoders import Nominatim

city = input("Enter your city name")

# find the coordinates using  geopy
address = '{}'.format(city)

geolocator = Nominatim(user_agent="cy_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,lat, lng))


Enter your city nameRome
The geograpical coordinate of Rome are 41.8933203, 12.4829321.


### So, we got the given city's coordinates and next step would be to get the data of that city using Foursquare API.

In [2]:
import json
from pandas.io.json import json_normalize

CLIENT_ID = 'IM0TJ0DO2AWOEENTZTCHLZ4DJXWXXY1ELFMC0U2A5O5ASGHQ'
CLIENT_SECRET = 'O1JWC1HPJKWMVWFYXFSFHGLDZQK20ENDWTX4UTKK3C0FVWSK'
VERSION = '20180724'
radius = 1000
LIMIT = 500
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
       CLIENT_ID, 
       CLIENT_SECRET, 
       VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

results = requests.get(url).json()

results

{'meta': {'code': 200, 'requestId': '60ec826f8941873f62996e7a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': 'With specials', 'key': 'specials'}]},
  'headerLocation': 'Rome',
  'headerFullLocation': 'Rome',
  'headerLocationGranularity': 'city',
  'totalResults': 242,
  'suggestedBounds': {'ne': {'lat': 41.90232030900001,
    'lng': 12.494999984048635},
   'sw': {'lat': 41.88432029099999, 'lng': 12.470864215951364}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcdac7f964a5209f5321e3',
       'name': 'Piazza del Campidoglio',
       'location': {'address': 'Piazza del Campidoglio',
        'lat': 41.89332137812433,
        'lng': 12.482955747846482,
        'labeledLatLngs': [{'lab

#### We get the data now we have to make a dataframe containing the venus and their categories along with their latitudes and longitudes.
#### First to get the category of each venue we will create a function and using this function we assing the category to every venue.

In [3]:
## to find the category of given venue

def get_category(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if(len(categories_list)==0):
        return Noe
    else:
        return categories_list[0]['name']

In [4]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)  #flatten Json

#filter columns
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category, axis=1)

#clean column
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-4-f32cfd4b8f56>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)  #flatten Json


,name,categories,lat,lng
0,Piazza del Campidoglio,Plaza,41.893321,12.482956
1,Foro Romano,Historic Site,41.892824,12.485503
2,Foro di Cesare,Historic Site,41.894128,12.485232
3,Foro di Traiano,Historic Site,41.894729,12.484871
4,Terrazza delle Quadrighe,Scenic Lookout,41.894346,12.483336


#### As we get the dataframe containing our requirements, let's look at the dataframe and the venues and their categories to have an idea what are we gonna see on our upcoming map.

In [5]:
# nearby_venues.shape
print('There are {} uniques venues.'.format(len(nearby_venues['name'].unique())))

There are 100 uniques venues.


#### Now it is time to create map using our dataframe without filtering of venue categories and see our selected city and the amount of venues we got.

In [85]:
mapCity = folium.Map()

for name,lat,lng in zip(nearby_venues['name'],nearby_venues['lat'],nearby_venues['lng']):
    
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='black',
                       fill=True,
                       fill_color='black',
                       fill_opacity=0.6,
                       parse_html=False).add_to(mapCity)
    
mapCity

#### Use filtering on our dataframe and creating new dataframe for applying K-Means Cluster method as it only works for numeric type of variables not for categorical or object type variables. so we make a dataframe having each category as column and having the value of 1 or 0 if they belong to particular venue name, based on these numbers our clusters will be made as if the venue name have value 0 or 1 for particular type of category

In [87]:
from sklearn.cluster import KMeans

In [95]:
Method_2 = pd.get_dummies(nearby_venues[['categories']], prefix="",prefix_sep="")

# add name column back to dataframe
Method_2['Venue Name'] = nearby_venues['name']

# move name column to the first column
fixed_columns = [Method_2.columns[-1]] + list(Method_2.columns[:-1])
Method_2 = Method_2[fixed_columns]

Method_grouped = Method_2.groupby('Venue Name').mean().reset_index()

Method_cluster = Method_2.drop('Venue Name',1)
kClusters = 8
kmeans_2 = KMeans(n_clusters = kClusters,random_state =0).fit(Method_cluster)

Method_grouped.insert(0,'Cluster labels',kmeans_2.labels_)
Method_grouped.head()

,Cluster labels,Venue Name,Art Museum,Bakery,Bar,Beer Bar,Bistro,Bookstore,Brewery,Café,...,Road,Roman Restaurant,Sandwich Place,Scenic Lookout,Sushi Restaurant,Temple,Theater,Toy / Game Store,Trattoria/Osteria,Wine Bar
0,3,Ai Tre Scalini,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,Al Vino! Al Vino!,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,Analemma Cafe,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,"Angelino ""ai Fori"" dal 1947",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,Antica Salumeria,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


#### Now we have our particular cluster label for each venue name, we just add category name and coordinates column of those venues which helps us in putting these venues on our  map 

In [98]:
Method_merged = nearby_venues

Method_merged.insert(0,'Cluster Labels',kmeans_2.labels_)

Method_merged.head()

,Cluster Labels,name,categories,lat,lng
0,3,Piazza del Campidoglio,Plaza,41.893321,12.482956
1,2,Foro Romano,Historic Site,41.892824,12.485503
2,2,Foro di Cesare,Historic Site,41.894128,12.485232
3,2,Foro di Traiano,Historic Site,41.894729,12.484871
4,0,Terrazza delle Quadrighe,Scenic Lookout,41.894346,12.483336


### Finally, after data scraping, data wrangling and our exploratory data analysis, and applying k-means clustering mechanism we are all set to see our results on map.

In [99]:
# create map
map_clusters = folium.Map(location = [lat,lng], zoom_start=0)

# set color scheme for the clusters
x = np.arange(kClusters)
ys = [i + x + (i*x)**2 for i in range(kClusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
markers_colors=[]
for lat,lng,name,cluster in zip(Method_merged['lat'], Method_merged['lng'], Method_merged['name'], Method_merged['Cluster Labels']):
    label = folium.Popup(str(name)+', Cluster'+str(int(cluster)), parse_html=True)
    cluster=int(cluster)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill=True,
        fil_color = rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    

map_clusters

#### Only by looking on map one can't find out, what a particular cluster belongs to or what are they representing apart from venue name, so for that we will analyze each cluster.

In [75]:
Method_merged.loc[Method_merged['Cluster Labels'] == 0]

,Cluster Labels,name,categories,lat,lng
4,0,Terrazza delle Quadrighe,Scenic Lookout,41.894346,12.483336
6,0,Capitoline Hill (Campidoglio),Scenic Lookout,41.893462,12.483588
7,0,Musei Capitolini,Art Museum,41.893343,12.482885
9,0,Teatro Della Cometa,Theater,41.893399,12.481593
19,0,Via dei Fori Imperiali,Road,41.894036,12.487228
20,0,Farnese Gardens,Garden,41.889928,12.485716
21,0,Mercati di Traiano - Museo dei Fori Imperiali,History Museum,41.896288,12.485973
24,0,Fontana delle Tartarughe,Fountain,41.893849,12.477514
30,0,PanePiù Bistrot,Sandwich Place,41.895067,12.477221
31,0,Pizza e Mozzarella,Pizza Place,41.897598,12.479097


#### Our cluster-0 shows some random venues together there is not more venues common.

In [77]:
Method_merged.loc[Method_merged['Cluster Labels'] == 1]

,Cluster Labels,name,categories,lat,lng
25,1,La Taverna dei Fori Imperiali,Italian Restaurant,41.893989,12.488528
29,1,"Angelino ""ai Fori"" dal 1947",Italian Restaurant,41.893154,12.487968
39,1,Nonna Betta,Italian Restaurant,41.892683,12.477956
41,1,Locanda del Prosciutto,Italian Restaurant,41.897230,12.478640
46,1,Vecchia Roma,Italian Restaurant,41.893379,12.479301
69,1,"Ristorante Tipico ""Tavernelle""",Italian Restaurant,41.896354,12.490539
72,1,Spirito Divino,Italian Restaurant,41.887880,12.476806
81,1,Pastachef,Italian Restaurant,41.894749,12.489673
92,1,Sette Oche,Italian Restaurant,41.888511,12.476487


#### Cluster-1 shows all the Italian Restaurant of the city.

In [78]:
Method_merged.loc[Method_merged['Cluster Labels'] == 2]

,Cluster Labels,name,categories,lat,lng
1,2,Foro Romano,Historic Site,41.892824,12.485503
2,2,Foro di Cesare,Historic Site,41.894128,12.485232
3,2,Foro di Traiano,Historic Site,41.894729,12.484871
12,2,Teatro di Marcello,Historic Site,41.891931,12.479798
13,2,Foro di Augusto,Historic Site,41.894070,12.486483
15,2,Foro della Pace,Historic Site,41.894288,12.484962
18,2,Portico d'Ottavia,Historic Site,41.892382,12.478500
22,2,Palazzo Valentini,Historic Site,41.896691,12.483806
32,2,Palazzo Doria Pamphili,Historic Site,41.897847,12.481534
43,2,Palatino,Historic Site,41.888234,12.487209


#### Cluster-2 shows all Historic sites of the city.

In [79]:
Method_merged.loc[Method_merged['Cluster Labels'] == 3]

,Cluster Labels,name,categories,lat,lng
0,3,Piazza del Campidoglio,Plaza,41.893321,12.482956
5,3,Piazza Venezia,Plaza,41.895747,12.482446
23,3,Piazza dell'Ara Coeli,Plaza,41.894426,12.481195
38,3,Largo di Torre Argentina,Plaza,41.895797,12.476852
40,3,Piazza di San Marco,Plaza,41.895529,12.481829
61,3,Piazza della Madonna dei Monti,Plaza,41.894963,12.490825
70,3,Piazza del Quirinale,Plaza,41.899295,12.486821
76,3,Piazza della Minerva,Plaza,41.897913,12.477421
86,3,Piazza di Pietra,Plaza,41.900053,12.479294
87,3,Piazza della Rotonda,Plaza,41.899253,12.476779


#### Cluster-3 shows all of the Plaza/shoping centers of the city, 

In [80]:
Method_merged.loc[Method_merged['Cluster Labels'] == 4]

,Cluster Labels,name,categories,lat,lng
10,4,Zuil van Trajanus (Colonna Traiana),Monument / Landmark,41.895816,12.484276
14,4,Palazzo Venezia,Monument / Landmark,41.895965,12.481852
27,4,Arco di Giano,Monument / Landmark,41.889333,12.482345
28,4,Arco di Tito,Monument / Landmark,41.890649,12.488537
71,4,Arco di Costantino,Monument / Landmark,41.889755,12.490618
79,4,Pantheon,Monument / Landmark,41.899133,12.476805


#### Cluster-4 shows the Monuments/Landmarks of the city.

In [81]:
Method_merged.loc[Method_merged['Cluster Labels'] == 5]

,Cluster Labels,name,categories,lat,lng
26,5,Wonderful Ice Cream,Ice Cream Shop,41.896833,12.487053
64,5,Venchi,Ice Cream Shop,41.900042,12.480883
68,5,Gelateria dell'Angeletto,Ice Cream Shop,41.894815,12.491431
83,5,Gelateria ai Cerchi,Ice Cream Shop,41.887864,12.483711
89,5,Gelateria del Viale,Ice Cream Shop,41.889903,12.474185


#### Cluster-5 shows the ice-cream shops in the city.

In [82]:
Method_merged.loc[Method_merged['Cluster Labels'] == 6]

,Cluster Labels,name,categories,lat,lng
16,6,Santa Maria in Campitelli,Church,41.893243,12.479585
45,6,San Marcello al Corso,Church,41.898594,12.481446
65,6,Basilica di Santa Maria sopra Minerva,Church,41.897892,12.477572
75,6,Basilica di Sant'Andrea della Valle,Church,41.896280,12.474383
99,6,Basilica di Santa Maria in Cosmedin,Church,41.888242,12.481382


#### Cluster-6 shows the Churchs in the city.

In [83]:
Method_merged.loc[Method_merged['Cluster Labels'] == 7]

,Cluster Labels,name,categories,lat,lng
8,7,Tempio di Vespasiano e Tito,Temple,41.892494,12.483688
11,7,Templio de Antonino e Faustina,Temple,41.892179,12.486694
17,7,Tempio del Divo Romolo,Temple,41.891863,12.487126
42,7,Tempio di Ercole Vincitore,Temple,41.888717,12.480796


#### Cluster-7 the last cluster shows the Temples in the city.

### By looking on the cluster distribution in the city, we can clearly say that the choosen city is a more of a culturla city and more towards it's historic places and monuments which makes it a great tourist place. 
### The one improvement I would like to do on my this project is creating a dashboard where you can get the city name as input from user and which shows them the map containing the cluster and the results of the clusters.